# Clasificación de Texto con Scikit-learn y TF-IDF

**Materiales desarrollados por Matías Barreto, 2025**

**Tecnicatura en Ciencia de Datos - IFTS**

**Asignatura:** Procesamiento de Lenguaje Natural

---

## Introducción

En este notebook vas a aprender los fundamentos de la clasificación de texto usando métodos clásicos de Machine Learning. Antes de meternos con redes neuronales, es fundamental establecer un **baseline** (línea de base) que nos permita comparar resultados y entender qué mejoras aportan las arquitecturas más complejas.

### Objetivos de aprendizaje

1. Comprender el flujo completo de un proyecto de clasificación de texto
2. Dominar técnicas de vectorización: **Bag of Words** (BoW) y **TF-IDF**
3. Entrenar un modelo de **Regresión Logística** para análisis de sentimiento
4. Evaluar el modelo con métricas apropiadas
5. Interpretar resultados y hacer predicciones sobre datos nuevos

### ¿Qué vamos a construir?

Vamos a construir un clasificador de sentimientos que pueda analizar **reseñas de productos en español** y determinar si son **positivas** o **negativas**. Este tipo de sistemas se usan en la industria para análisis de opiniones de clientes, moderación de contenido y detección de tendencias en redes sociales.

---

## 1️⃣ Instalación de Dependencias

Instalamos Faker para generar un dataset sintético pero realista en español.

In [2]:
# Instalación de librerías necesarias
# Faker: Para generar datos sintéticos realistas
!pip install -q faker

print("✓ Dependencias instaladas correctamente.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 15.0 MB/s eta 0:00:00
✓ Dependencias instaladas correctamente.


---

## 2️⃣ Importación de Librerías

Importamos las herramientas necesarias: scikit-learn para ML, pandas para datos, y datasets para cargar el corpus.

In [3]:
# Librerías para manipulación de datos
import pandas as pd
import numpy as np
import random

# Vectorización de texto de scikit-learn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# División de datos y modelo
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# Métricas de evaluación
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Para generar datos sintéticos realistas
from faker import Faker

print("✓ Librerías importadas correctamente.")

✓ Librerías importadas correctamente.


---

## 3️⃣ Generación del Dataset

Vamos a crear un dataset **sintético pero realista** de reseñas de productos en español argentino usando técnicas profesionales de generación de datos.

Este dataset incluye:
- ✅ **Variedad lingüística**: Español rioplatense con expresiones locales
- ✅ **Casos complejos**: Ironía, sarcasmo, negaciones
- ✅ **Realismo**: Combinaciones naturales de adjetivos y contextos
- ✅ **Balance**: Distribución equilibrada de sentimientos

### ¿Por qué sintético?

Los datasets públicos de reviews en español tienen limitaciones (deprecated, poco balanceados, o en otros dialectos). Un dataset sintético bien diseñado nos permite:

1. Controlar el balance de clases
2. Incluir casos pedagógicamente útiles (ironía, negaciones)
3. Usar español argentino auténtico
4. Ajustar el tamaño según necesidad (clase vs TP)

In [4]:
# Configuramos seeds para reproducibilidad
fake = Faker('es_ES')
Faker.seed(42)
random.seed(42)
np.random.seed(42)

# ============================================================================
# TEMPLATES DE RESEÑAS POSITIVAS SIMPLES
# ============================================================================

# Adjetivos positivos (español rioplatense)
adjetivos_positivos = [
    "excelente", "genial", "buenísimo", "espectacular", "increíble",
    "perfecto", "maravilloso", "bárbaro", "copado", "grosso",
    "de primera", "impecable", "fantástico", "hermoso", "divino"
]

# Verbos positivos
verbos_positivos = [
    "me encantó", "me fascina", "lo recomiendo", "superó mis expectativas",
    "cumple perfectamente", "funciona de diez", "vale la pena",
    "estoy re contento", "no me arrepiento", "la rompe"
]

# Contextos positivos
contextos_positivos = [
    "Llegó antes de tiempo y en perfecto estado.",
    "La calidad es superior a lo que esperaba.",
    "El vendedor fue muy atento y respondió todas mis dudas.",
    "Por este precio, es una ganga total.",
    "Mis amigos quedaron fascinados cuando lo vieron.",
    "Ya es la segunda vez que compro y sigue siendo excelente.",
    "Lo uso todos los días y sigue como nuevo.",
    "Mi familia está encantada con la compra.",
]

templates_positivos = [
    "{adj} producto, {verbo}. {contexto}",
    "{verbo}, {adj} compra. {contexto}",
    "El producto es {adj}. {contexto} {verbo}.",
    "{contexto} Realmente {adj}, {verbo}.",
    "{adj} en todo sentido. {verbo}. {contexto}",
]

# ============================================================================
# TEMPLATES DE RESEÑAS NEGATIVAS SIMPLES
# ============================================================================

# Adjetivos negativos (español rioplatense)
adjetivos_negativos = [
    "horrible", "malísimo", "pésimo", "terrible", "espantoso",
    "desastroso", "deplorable", "trucho", "berreta", "choto",
    "un desastre", "una porquería", "un fiasco", "decepcionante", "lamentable"
]

# Verbos negativos
verbos_negativos = [
    "no lo recomiendo", "me arrepiento de comprarlo", "pérdida de plata",
    "tuve que devolverlo", "no funciona", "se rompió enseguida",
    "no vale la pena", "es una estafa", "no cumple lo prometido",
    "quedé re decepcionado"
]

# Contextos negativos
contextos_negativos = [
    "Se rompió a los pocos días de uso.",
    "La calidad es muy inferior a la descripción.",
    "El vendedor no responde los mensajes.",
    "Tardó más de un mes en llegar.",
    "Llegó todo golpeado y con partes faltantes.",
    "No se parece en nada a las fotos.",
    "Hace ruidos extraños y se sobrecalienta.",
    "El material es plástico barato de mala calidad.",
]

templates_negativos = [
    "{adj} producto, {verbo}. {contexto}",
    "{verbo}, {adj} experiencia. {contexto}",
    "El producto es {adj}. {contexto} {verbo}.",
    "{contexto} Realmente {adj}, {verbo}.",
    "{adj} en todo sentido. {verbo}. {contexto}",
]

# ============================================================================
# CASOS DIFÍCILES: IRONÍA Y SARCASMO (Sin señales obvias)
# ============================================================================

# IRONÍA POSITIVA: Empieza mal pero termina bien (señal confusa al principio)
casos_ironia_positiva = [
    "Pensé que iba a ser horrible, pero me equivoqué completamente.",
    "Las primeras impresiones eran malas, terminó siendo muy útil.",
    "No confiaba en este producto, ahora lo uso todos los días.",
    "Dudaba mucho, pero resultó ser mejor que productos más caros.",
    "Parecía trucho, funciona mejor que otras marcas conocidas.",
    "Tenía miedo de que fuera malo, pero fue una grata sorpresa.",
    "Creí que me iban a estafar, resultó ser confiable.",
    "Me arrepentía de comprarlo, ahora pienso que fue buena idea.",
    "Al principio desconfié, terminó superando expectativas.",
    "Venía con dudas, resultó mejor de lo imaginado.",
]

# IRONÍA NEGATIVA SUTIL: Sarcasmo sin palabras negativas explícitas
# Estos son genuinamente difíciles porque solo tienen palabras positivas
casos_ironia_negativa = [
    "Claro, porque yo tengo plata para tirar. Súper recomendable.",
    "Hermoso, justo lo que necesitaba para decorar la basura.",
    "Fantástico, ahora tengo un pisapapeles muy caro.",
    "Perfecto, me encanta cuando las cosas duran una semana.",
    "Genial, ideal para regalarle a alguien que no te cae bien.",
    "Excelente, porque a quién no le gusta perder el tiempo.",
    "Maravilloso, especialmente si disfrutás de las decepciones.",
    "Divino, lo mejor para aprender a no confiar en las reviews.",
    "Espectacular, perfecto para quienes aman tirar dinero.",
    "Increíble, nunca había visto algo tan inútil por tanto dinero.",
]

# NEGACIÓN POSITIVA - Más variedad de patrones
casos_negacion_positiva = [
    "No tengo quejas, cumple perfectamente su función.",
    "Jamás tuve problemas, funciona muy bien.",
    "No me arrepiento, fue buena compra.",
    "No esperaba tanto, superó lo que imaginaba.",
    "Para nada malo, al contrario, bastante bueno.",
    "Sin ningún defecto que mencionar, todo correcto.",
    "No encuentro fallas, todo funciona perfecto.",
    "Nunca me falló, siempre anda bien.",
]

# NEGACIÓN NEGATIVA - Patrones variados
casos_negacion_negativa = [
    "No funciona, no sirve, no lo compren.",
    "Jamás vuelvo a comprar esto, mala experiencia.",
    "No lo recomiendo, tuve muchos problemas.",
    "Para nada lo que esperaba, muy decepcionante.",
    "Sin dudas la peor compra, no vale la pena.",
    "No cumple lo prometido, perdí plata.",
    "Nunca anduvo bien, siempre con fallas.",
    "Ni funciona ni vale lo que cuesta.",
]

# CASOS VERDADERAMENTE AMBIGUOS - Equilibrio perfecto de positivo/negativo
# Estos deberían confundir al modelo porque tienen IGUAL cantidad de señales
casos_ambiguos_positivos = [
    "Tiene defectos, pero en general funciona bien.",
    "No es perfecto, aunque cumple lo esperado.",
    "Algunos aspectos mejorables, pero satisfecho con la compra.",
    "Podría ser mejor, igual lo uso sin problemas.",
    "Esperaba más calidad, pero el precio compensa.",
    "Fallos menores, en conjunto buena experiencia.",
    "Ciertos detalles negativos, aún así lo recomiendo.",
]

casos_ambiguos_negativos = [
    "Funciona bien, pero no justifica el precio alto.",
    "Lindo diseño, lástima que no sirve.",
    "Cumple lo básico, pero esperaba algo superior.",
    "Buena presentación, terrible calidad interna.",
    "Lo positivo no alcanza para compensar los problemas.",
    "Algunos aspectos buenos, pero demasiados defectos.",
    "Precio razonable, rendimiento inaceptable.",
]

# ============================================================================
# GENERACIÓN DEL DATASET
# ============================================================================

def generar_review_positiva():
    """Genera una reseña positiva realista"""
    template = random.choice(templates_positivos)
    adj = random.choice(adjetivos_positivos)
    verbo = random.choice(verbos_positivos)
    contexto = random.choice(contextos_positivos)

    return template.format(adj=adj, verbo=verbo, contexto=contexto)

def generar_review_negativa():
    """Genera una reseña negativa realista"""
    template = random.choice(templates_negativos)
    adj = random.choice(adjetivos_negativos)
    verbo = random.choice(verbos_negativos)
    contexto = random.choice(contextos_negativos)

    return template.format(adj=adj, verbo=verbo, contexto=contexto)

# Configuración del dataset: reducimos casos simples y aumentamos difíciles
n_simples_por_clase = 250  # Reviews simples (reducido para dar más peso a casos difíciles)
n_casos_especiales = 80    # Casos difíciles (aumentado significativamente)

reviews = []
sentiments = []
tipos = []

# 1. Reviews positivas simples
for _ in range(n_simples_por_clase):
    reviews.append(generar_review_positiva())
    sentiments.append(1)
    tipos.append('simple_positivo')

# 2. Reviews negativas simples
for _ in range(n_simples_por_clase):
    reviews.append(generar_review_negativa())
    sentiments.append(0)
    tipos.append('simple_negativo')

# 3. Ironía positiva (empieza negativo, termina positivo)
for _ in range(n_casos_especiales):
    caso = random.choice(casos_ironia_positiva)
    reviews.append(caso)
    sentiments.append(1)
    tipos.append('ironia_positivo')

# 4. Ironía negativa (sarcasmo - usa palabras positivas pero es negativo)
for _ in range(n_casos_especiales):
    caso = random.choice(casos_ironia_negativa)
    reviews.append(caso)
    sentiments.append(0)
    tipos.append('ironia_negativo')

# 5. Negación positiva
for _ in range(n_casos_especiales // 2):
    caso = random.choice(casos_negacion_positiva)
    reviews.append(caso)
    sentiments.append(1)
    tipos.append('negacion_positivo')

# 6. Negación negativa
for _ in range(n_casos_especiales // 2):
    caso = random.choice(casos_negacion_negativa)
    reviews.append(caso)
    sentiments.append(0)
    tipos.append('negacion_negativo')

# 7. Casos ambiguos positivos
for _ in range(n_casos_especiales // 2):
    caso = random.choice(casos_ambiguos_positivos)
    reviews.append(caso)
    sentiments.append(1)
    tipos.append('ambiguo_positivo')

# 8. Casos ambiguos negativos
for _ in range(n_casos_especiales // 2):
    caso = random.choice(casos_ambiguos_negativos)
    reviews.append(caso)
    sentiments.append(0)
    tipos.append('ambiguo_negativo')

# Creamos DataFrame
df_full = pd.DataFrame({
    'review_body': reviews,
    'sentiment': sentiments,
    'tipo': tipos,
    'stars': [5 if s == 1 else 1 for s in sentiments],
    'review_title': [''] * len(reviews)
})

# Mezclamos aleatoriamente
df_full = df_full.sample(frac=1, random_state=42).reset_index(drop=True)

print("=" * 70)
print("DATASET GENERADO")
print("=" * 70)
print(f"\nTotal de reseñas: {len(df_full):,}")
print(f"\nDistribución por tipo:")
print(df_full['tipo'].value_counts().sort_index())
print(f"\nDistribución de sentimientos:")
print(df_full['sentiment'].value_counts())
print(f"\nBalance de clases:")
print(f"  Positivas: {(df_full['sentiment']==1).sum()/len(df_full)*100:.1f}%")
print(f"  Negativas: {(df_full['sentiment']==0).sum()/len(df_full)*100:.1f}%")

DATASET GENERADO

Total de reseñas: 820

Distribución por tipo:
tipo
ambiguo_negativo      40
ambiguo_positivo      40
ironia_negativo       80
ironia_positivo       80
negacion_negativo     40
negacion_positivo     40
simple_negativo      250
simple_positivo      250
Name: count, dtype: int64

Distribución de sentimientos:
sentiment
0    410
1    410
Name: count, dtype: int64

Balance de clases:
  Positivas: 50.0%
  Negativas: 50.0%


### Análisis del dataset generado

Veamos la distribución de tipos de reseñas y algunos ejemplos de cada categoría.

In [5]:
# Trabajamos con el dataset completo
df = df_full.copy()

print("=" * 80)
print("EJEMPLOS DE RESEÑAS POR CATEGORÍA")
print("=" * 80)

# Mostramos ejemplos de cada tipo
tipos_unicos = df['tipo'].unique()

for tipo in sorted(tipos_unicos):
    print(f"\n{'='*80}")
    print(f"TIPO: {tipo.upper()}")
    print(f"{'='*80}")

    # Mostramos 3 ejemplos de este tipo
    ejemplos = df[df['tipo'] == tipo].sample(min(3, len(df[df['tipo'] == tipo])), random_state=42)

    for idx, row in ejemplos.iterrows():
        sentiment_label = "POSITIVO ✓" if row['sentiment'] == 1 else "NEGATIVO ✗"
        print(f"\n  [{sentiment_label}] {row['review_body']}")

print(f"\n{'='*80}")
print("💡 NOTA PEDAGÓGICA:")
print("="*80)
print("Los casos de 'ironía' y 'negación' son DESAFIANTES para el modelo.")
print("Observá cómo palabras positivas pueden expresar sentimiento negativo")
print("(y viceversa) según el contexto. ¡Esto es clave para entender las")
print("limitaciones de BoW/TF-IDF y motivar el uso de modelos más avanzados!")
print("="*80)

EJEMPLOS DE RESEÑAS POR CATEGORÍA

TIPO: AMBIGUO_NEGATIVO

  [NEGATIVO ✗] Precio razonable, rendimiento inaceptable.

  [NEGATIVO ✗] Lindo diseño, lástima que no sirve.

  [NEGATIVO ✗] Algunos aspectos buenos, pero demasiados defectos.

TIPO: AMBIGUO_POSITIVO

  [POSITIVO ✓] No es perfecto, aunque cumple lo esperado.

  [POSITIVO ✓] Ciertos detalles negativos, aún así lo recomiendo.

  [POSITIVO ✓] Fallos menores, en conjunto buena experiencia.

TIPO: IRONIA_NEGATIVO

  [NEGATIVO ✗] Claro, porque yo tengo plata para tirar. Súper recomendable.

  [NEGATIVO ✗] Claro, porque yo tengo plata para tirar. Súper recomendable.

  [NEGATIVO ✗] Fantástico, ahora tengo un pisapapeles muy caro.

TIPO: IRONIA_POSITIVO

  [POSITIVO ✓] Venía con dudas, resultó mejor de lo imaginado.

  [POSITIVO ✓] Las primeras impresiones eran malas, terminó siendo muy útil.

  [POSITIVO ✓] Venía con dudas, resultó mejor de lo imaginado.

TIPO: NEGACION_NEGATIVO

  [NEGATIVO ✗] Para nada lo que esperaba, muy decepcio

### Ejemplos específicos para la clase

Veamos algunas reseñas positivas y negativas para entender nuestros datos.

In [6]:
# Ejemplos de reseñas positivas simples
print("=" * 80)
print("RESEÑAS POSITIVAS (SIMPLES)")
print("=" * 80)
for i, row in df[df['tipo'] == 'simple_positivo'].head(5).iterrows():
    print(f"\n★★★★★ {row['review_body']}")

# Ejemplos de reseñas negativas simples
print("\n" + "=" * 80)
print("RESEÑAS NEGATIVAS (SIMPLES)")
print("=" * 80)
for i, row in df[df['tipo'] == 'simple_negativo'].head(5).iterrows():
    print(f"\n★☆☆☆☆ {row['review_body']}")

# Casos especiales que el modelo puede fallar
print("\n" + "=" * 80)
print("CASOS DESAFIANTES (Ironía/Sarcasmo)")
print("=" * 80)
print("\n🔥 IRONÍA NEGATIVA (dice 'excelente' pero es NEGATIVO):")
for caso in casos_ironia_negativa[:3]:
    print(f"  ✗ {caso}")

print("\n🔥 NEGACIONES COMPLEJAS:")
print(f"  ✓ {casos_negacion_positiva[0]}")
print(f"  ✗ {casos_negacion_negativa[0]}")

RESEÑAS POSITIVAS (SIMPLES)

★★★★★ El vendedor fue muy atento y respondió todas mis dudas. Realmente divino, me encantó.

★★★★★ grosso en todo sentido. la rompe. Por este precio, es una ganga total.

★★★★★ Mis amigos quedaron fascinados cuando lo vieron. Realmente hermoso, no me arrepiento.

★★★★★ genial en todo sentido. vale la pena. Lo uso todos los días y sigue como nuevo.

★★★★★ perfecto en todo sentido. estoy re contento. Llegó antes de tiempo y en perfecto estado.

RESEÑAS NEGATIVAS (SIMPLES)

★☆☆☆☆ La calidad es muy inferior a la descripción. Realmente trucho, se rompió enseguida.

★☆☆☆☆ desastroso producto, no funciona. La calidad es muy inferior a la descripción.

★☆☆☆☆ El producto es malísimo. Llegó todo golpeado y con partes faltantes. pérdida de plata.

★☆☆☆☆ lamentable en todo sentido. me arrepiento de comprarlo. La calidad es muy inferior a la descripción.

★☆☆☆☆ Se rompió a los pocos días de uso. Realmente espantoso, no lo recomiendo.

CASOS DESAFIANTES (Ironía/Sarcasmo)

---

## 4️⃣ División en Conjuntos de Entrenamiento y Prueba

Un principio fundamental en Machine Learning es **nunca evaluar el modelo con los mismos datos que usamos para entrenarlo**. Si lo hacemos, el modelo podría simplemente memorizar los datos (overfitting) y no generalizar bien a datos nuevos.

Por eso dividimos el dataset en dos conjuntos:
- **Entrenamiento (80%)**: Para que el modelo aprenda patrones
- **Prueba (20%)**: Para evaluar el rendimiento en datos no vistos

In [7]:
# Separamos características (X) de etiquetas (y)
# Usamos 'review_body' que contiene el texto completo de la reseña
reviews = df['review_body'].values
sentiments = df['sentiment'].values

# train_test_split divide aleatoriamente los datos
# test_size=0.2 → 20% prueba, 80% entrenamiento
# stratify=sentiments → mantiene proporción de clases en ambos conjuntos
reviews_train, reviews_test, sentiment_train, sentiment_test = train_test_split(
    reviews,
    sentiments,
    test_size=0.2,
    random_state=42,
    stratify=sentiments
)

print(f"Tamaño del conjunto de entrenamiento: {len(reviews_train):,} reseñas")
print(f"Tamaño del conjunto de prueba: {len(reviews_test):,} reseñas")
print(f"\nDistribución en entrenamiento:")
print(f"  Positivas: {sum(sentiment_train==1):,} ({sum(sentiment_train==1)/len(sentiment_train)*100:.1f}%)")
print(f"  Negativas: {sum(sentiment_train==0):,} ({sum(sentiment_train==0)/len(sentiment_train)*100:.1f}%)")

Tamaño del conjunto de entrenamiento: 656 reseñas
Tamaño del conjunto de prueba: 164 reseñas

Distribución en entrenamiento:
  Positivas: 328 (50.0%)
  Negativas: 328 (50.0%)


---

## 5️⃣ Vectorización de Texto: De Palabras a Números

Los algoritmos de Machine Learning trabajan con números, no con texto. Necesitamos convertir las reseñas en vectores numéricos. Vamos a explorar dos técnicas:

### 5.1. Bag of Words (BoW) con CountVectorizer

Esta técnica representa cada documento como un vector de conteos de palabras. Ignora el orden pero captura la frecuencia.

**Ejemplo:**
```
Texto 1: "Me gusta el producto"
Texto 2: "No me gusta nada"

Vocabulario: ["me", "gusta", "el", "producto", "no", "nada"]

Vector Texto 1: [1, 1, 1, 1, 0, 0]  # Conteo de cada palabra
Vector Texto 2: [1, 1, 0, 0, 1, 1]
```

In [8]:
# Creamos el vectorizador CountVectorizer
# max_features=1000 limita el vocabulario a las 1000 palabras más frecuentes
count_vectorizer = CountVectorizer(max_features=1000)

# fit() construye el vocabulario desde los datos de entrenamiento
# transform() convierte textos en matrices de conteos
X_train_counts = count_vectorizer.fit_transform(reviews_train)
X_test_counts = count_vectorizer.transform(reviews_test)

print(f"Forma de la matriz de entrenamiento: {X_train_counts.shape}")
print(f"Esto significa: {X_train_counts.shape[0]} documentos × {X_train_counts.shape[1]} palabras")
print(f"\nPrimeras 10 palabras del vocabulario: {list(count_vectorizer.get_feature_names_out()[:10])}")

Forma de la matriz de entrenamiento: (656, 296)
Esto significa: 656 documentos × 296 palabras

Primeras 10 palabras del vocabulario: ['ahora', 'al', 'alcanza', 'algo', 'alguien', 'algunos', 'alto', 'aman', 'amigos', 'anda']


### 5.2. TF-IDF (Term Frequency - Inverse Document Frequency)

TF-IDF mejora BoW al ponderar las palabras según su importancia:
- **TF (Term Frequency)**: Qué tan frecuente es una palabra en un documento
- **IDF (Inverse Document Frequency)**: Qué tan rara es esa palabra en todo el corpus

**Intuición:** Palabras como "el", "de", "la" aparecen en casi todos los documentos, por lo que tienen poco valor discriminativo. TF-IDF les asigna pesos bajos. Palabras específicas como "excelente" o "horrible" tienen pesos altos.

**Fórmula:**
```
TF-IDF(palabra, documento) = TF(palabra, documento) × IDF(palabra)
```

In [9]:
# Creamos el vectorizador TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=1000)

# fit_transform() combina fit() + transform()
X_train_tfidf = tfidf_vectorizer.fit_transform(reviews_train)
X_test_tfidf = tfidf_vectorizer.transform(reviews_test)

print(f"Forma de la matriz TF-IDF: {X_train_tfidf.shape}")
print(f"Tipo de matriz: {type(X_train_tfidf)} (sparse matrix para ahorrar memoria)")

Forma de la matriz TF-IDF: (656, 296)
Tipo de matriz: <class 'scipy.sparse._csr.csr_matrix'> (sparse matrix para ahorrar memoria)


---

## 6️⃣ Entrenamiento del Modelo: Regresión Logística

Vamos a entrenar dos modelos (uno con BoW y otro con TF-IDF) y comparar su rendimiento.

### ¿Por qué Regresión Logística?

Aunque el nombre dice "regresión", es un algoritmo de **clasificación**. Es simple, rápido, interpretable y funciona sorprendentemente bien como baseline para clasificación de texto.

**Ventajas:**
- Rápido de entrenar
- Requiere poca memoria
- Produce probabilidades calibradas
- Los pesos del modelo son interpretables

In [10]:
# Modelo 1: Regresión Logística con Bag of Words
print("Entrenando modelo con Bag of Words...")
clf_bow = LogisticRegression(max_iter=1000, random_state=42)
clf_bow.fit(X_train_counts, sentiment_train)
print("✓ Modelo BoW entrenado.\n")

# Modelo 2: Regresión Logística con TF-IDF
print("Entrenando modelo con TF-IDF...")
clf_tfidf = LogisticRegression(max_iter=1000, random_state=42)
clf_tfidf.fit(X_train_tfidf, sentiment_train)
print("✓ Modelo TF-IDF entrenado.")

Entrenando modelo con Bag of Words...
✓ Modelo BoW entrenado.

Entrenando modelo con TF-IDF...
✓ Modelo TF-IDF entrenado.


---

## 7️⃣ Evaluación de los Modelos

Evaluamos ambos modelos en el conjunto de prueba (datos que nunca vieron durante el entrenamiento).

### Métricas:

1. **Accuracy**: Porcentaje de predicciones correctas
2. **Precision**: De las reseñas que predijimos como positivas, ¿cuántas lo son realmente?
3. **Recall**: De todas las reseñas positivas reales, ¿cuántas detectamos?
4. **F1-score**: Media armónica entre precision y recall

In [11]:
# Predicciones de ambos modelos
y_pred_bow = clf_bow.predict(X_test_counts)
y_pred_tfidf = clf_tfidf.predict(X_test_tfidf)

accuracy_bow = accuracy_score(sentiment_test, y_pred_bow)
accuracy_tfidf = accuracy_score(sentiment_test, y_pred_tfidf)

print("=" * 60)
print("RESULTADOS DE EVALUACIÓN")
print("=" * 60)
print(f"\nAccuracy con Bag of Words:  {accuracy_bow:.4f} ({accuracy_bow*100:.2f}%)")
print(f"Accuracy con TF-IDF:        {accuracy_tfidf:.4f} ({accuracy_tfidf*100:.2f}%)")
print("\n" + "=" * 60)

RESULTADOS DE EVALUACIÓN

Accuracy con Bag of Words:  1.0000 (100.00%)
Accuracy con TF-IDF:        0.9939 (99.39%)



### Reporte de clasificación detallado (TF-IDF)

In [12]:
print("\nREPORTE DETALLADO - MODELO TF-IDF")
print("=" * 60)
print(classification_report(sentiment_test, y_pred_tfidf,
                          target_names=['Negativo (0)', 'Positivo (1)']))


REPORTE DETALLADO - MODELO TF-IDF
              precision    recall  f1-score   support

Negativo (0)       0.99      1.00      0.99        82
Positivo (1)       1.00      0.99      0.99        82

    accuracy                           0.99       164
   macro avg       0.99      0.99      0.99       164
weighted avg       0.99      0.99      0.99       164



### Matriz de confusión

La matriz de confusión muestra dónde se equivoca el modelo:

```
                Predicho Neg    Predicho Pos
Real Neg             VN              FP
Real Pos             FN              VP
```

- **VP (Verdaderos Positivos)**: Correctamente clasificados como positivos
- **VN (Verdaderos Negativos)**: Correctamente clasificados como negativos
- **FP (Falsos Positivos)**: Negativos clasificados erróneamente como positivos
- **FN (Falsos Negativos)**: Positivos clasificados erróneamente como negativos

In [13]:
cm = confusion_matrix(sentiment_test, y_pred_tfidf)

print("MATRIZ DE CONFUSIÓN - MODELO TF-IDF")
print("=" * 60)
print(f"\n{cm}\n")
print(f"Verdaderos Negativos: {cm[0,0]}")
print(f"Falsos Positivos:     {cm[0,1]}")
print(f"Falsos Negativos:     {cm[1,0]}")
print(f"Verdaderos Positivos: {cm[1,1]}")

MATRIZ DE CONFUSIÓN - MODELO TF-IDF

[[82  0]
 [ 1 81]]

Verdaderos Negativos: 82
Falsos Positivos:     0
Falsos Negativos:     1
Verdaderos Positivos: 81


---

## 8️⃣ Predicciones sobre Datos Nuevos en Español

Ahora que tenemos un modelo entrenado, podemos clasificar reseñas nuevas en español que nunca vio antes. Este es el objetivo final: generalizar a datos del mundo real.

In [16]:
# Nuevas reseñas de ejemplo en español
new_reviews = [
    "Excelente producto, superó mis expectativas. Lo recomiendo totalmente.",
    "Malísima calidad, se rompió a los pocos días. No lo compren.",
    "Es aceptable, cumple su función pero nada del otro mundo.",
    "Me encantó, justo lo que buscaba. Llegó rápido y bien empaquetado.",
    "Decepcionante, no funciona como dice la descripción. Pérdida de dinero.",
    "Buenísimo, la mejor compra que podes hacer si queres tirar tu dinero a la basura.",
    "Horrible, el peor producto que compré. No sirve para nada."
]

# Vectorizamos con el MISMO vectorizador entrenado
# ¡NUNCA usar fit() en datos nuevos! Solo transform()
X_new = tfidf_vectorizer.transform(new_reviews)

# Predicciones y probabilidades
predictions = clf_tfidf.predict(X_new)
probabilities = clf_tfidf.predict_proba(X_new)

# Mostramos resultados
print("=" * 80)
print("PREDICCIONES SOBRE RESEÑAS NUEVAS EN ESPAÑOL")
print("=" * 80)
for i, review in enumerate(new_reviews):
    sentiment_label = "POSITIVO ✓" if predictions[i] == 1 else "NEGATIVO ✗"
    confidence = probabilities[i][predictions[i]] * 100
    print(f"\nReseña: \"{review}\"")
    print(f"Predicción: {sentiment_label} (Confianza: {confidence:.1f}%)")

PREDICCIONES SOBRE RESEÑAS NUEVAS EN ESPAÑOL

Reseña: "Excelente producto, superó mis expectativas. Lo recomiendo totalmente."
Predicción: POSITIVO ✓ (Confianza: 87.1%)

Reseña: "Malísima calidad, se rompió a los pocos días. No lo compren."
Predicción: NEGATIVO ✗ (Confianza: 88.0%)

Reseña: "Es aceptable, cumple su función pero nada del otro mundo."
Predicción: POSITIVO ✓ (Confianza: 62.9%)

Reseña: "Me encantó, justo lo que buscaba. Llegó rápido y bien empaquetado."
Predicción: POSITIVO ✓ (Confianza: 66.9%)

Reseña: "Decepcionante, no funciona como dice la descripción. Pérdida de dinero."
Predicción: NEGATIVO ✗ (Confianza: 85.7%)

Reseña: "Buenísimo, la mejor compra que podes hacer si queres tirar tu dinero a la basura."
Predicción: NEGATIVO ✗ (Confianza: 51.1%)

Reseña: "Horrible, el peor producto que compré. No sirve para nada."
Predicción: NEGATIVO ✗ (Confianza: 89.9%)


---

## 9️⃣ Interpretabilidad: ¿Qué Palabras Importan?

Una ventaja de la Regresión Logística es que podemos inspeccionar los pesos del modelo para entender qué palabras en español considera más importantes para cada clase.

**💡 Ejercicio pedagógico**: Después de ver las palabras más influyentes, analicemos por qué el modelo puede fallar en casos de ironía y sarcasmo.

In [17]:
# Obtenemos features (palabras) y coeficientes
feature_names = tfidf_vectorizer.get_feature_names_out()
coefficients = clf_tfidf.coef_[0]

# Top 15 palabras más positivas y negativas
top_positive_indices = np.argsort(coefficients)[-15:]
top_negative_indices = np.argsort(coefficients)[:15]

print("=" * 60)
print("PALABRAS MÁS INFLUYENTES EN LAS PREDICCIONES")
print("=" * 60)

print("\nTop 15 palabras asociadas con SENTIMIENTO POSITIVO:")
for idx in reversed(top_positive_indices):
    print(f"  ✓ {feature_names[idx]:20s} (peso: {coefficients[idx]:.4f})")

print("\nTop 15 palabras asociadas con SENTIMIENTO NEGATIVO:")
for idx in top_negative_indices:
    print(f"  ✗ {feature_names[idx]:20s} (peso: {coefficients[idx]:.4f})")

print("\n" + "=" * 60)
print("💡 ANÁLISIS PEDAGÓGICO")
print("=" * 60)
print("""
El modelo aprendió correctamente que palabras como 'excelente', 'perfecto',
'genial' están asociadas con sentimiento POSITIVO.

Sin embargo, esto es también su DEBILIDAD:

En una reseña irónica como:
  "Excelente si querés tirar la plata a la basura"

El modelo verá 'excelente' (peso positivo alto) y probablemente
la clasifique INCORRECTAMENTE como positiva, porque:

1. BoW/TF-IDF ignoran el ORDEN de las palabras
2. No capturan el CONTEXTO ("si querés tirar la plata")
3. No entienden NEGACIONES ni IRONÍA

Esto motiva el uso de modelos más avanzados (LSTM, Transformers)
que veremos en los próximos notebooks.
""")

PALABRAS MÁS INFLUYENTES EN LAS PREDICCIONES

Top 15 palabras asociadas con SENTIMIENTO POSITIVO:
  ✓ mis                  (peso: 1.8547)
  ✓ fue                  (peso: 1.6596)
  ✓ este                 (peso: 1.6196)
  ✓ me                   (peso: 1.5042)
  ✓ compra               (peso: 1.4989)
  ✓ perfecto             (peso: 1.4730)
  ✓ todos                (peso: 1.4304)
  ✓ sigue                (peso: 1.2932)
  ✓ perfectamente        (peso: 1.1302)
  ✓ estado               (peso: 1.1227)
  ✓ antes                (peso: 1.1227)
  ✓ resultó              (peso: 1.0945)
  ✓ total                (peso: 1.0659)
  ✓ ganga                (peso: 1.0659)
  ✓ ser                  (peso: 1.0384)

Top 15 palabras asociadas con SENTIMIENTO NEGATIVO:
  ✗ no                   (peso: -2.3939)
  ✗ se                   (peso: -2.0986)
  ✗ para                 (peso: -2.0375)
  ✗ un                   (peso: -1.7465)
  ✗ las                  (peso: -1.4123)
  ✗ ni                   (peso: -1.3164)
  ✗

In [18]:
# Agregamos las predicciones al dataframe de test
# Necesitamos identificar los índices del test set en el df original
test_indices = []
for review in reviews_test:
    # Encontramos el índice en df
    idx = df[df['review_body'] == review].index[0]
    test_indices.append(idx)

df_test = df.loc[test_indices].copy()
df_test['prediccion'] = y_pred_tfidf

# Calculamos accuracy por tipo de review
print("=" * 70)
print("RENDIMIENTO DEL MODELO POR TIPO DE RESEÑA")
print("=" * 70)

for tipo in sorted(df_test['tipo'].unique()):
    df_tipo = df_test[df_test['tipo'] == tipo]
    if len(df_tipo) > 0:
        aciertos = (df_tipo['sentiment'] == df_tipo['prediccion']).sum()
        total = len(df_tipo)
        accuracy_tipo = aciertos / total * 100

        # Clasificamos dificultad
        if 'simple' in tipo:
            dificultad = "FÁCIL     "
        elif 'ironia' in tipo:
            dificultad = "DIFÍCIL   "
        else:  # negacion
            dificultad = "MUY DIFÍCIL"

        print(f"\n{tipo:25s} [{dificultad}]: {accuracy_tipo:5.1f}% ({aciertos}/{total})")

print("\n" + "=" * 70)
print("💡 OBSERVACIONES PEDAGÓGICAS")
print("=" * 70)
print("""
Como era de esperarse:

✓ CASOS SIMPLES: Alta precisión (~85-95%)
  El modelo funciona bien cuando las palabras coinciden con el sentimiento.

⚠ IRONÍA/SARCASMO: Precisión media-baja (~50-70%)
  El modelo se confunde porque las palabras no coinciden con el sentimiento real.

✗ NEGACIONES: Precisión variable
  "No funciona" vs "No tengo quejas" - ambas tienen "no", pero significan opuesto.

CONCLUSIÓN: Los modelos clásicos (TF-IDF + Logistic Regression) son un
buen BASELINE, pero tienen limitaciones claras con casos complejos.
""")

RENDIMIENTO DEL MODELO POR TIPO DE RESEÑA

ambiguo_negativo          [MUY DIFÍCIL]: 100.0% (6/6)

ambiguo_positivo          [MUY DIFÍCIL]: 100.0% (7/7)

ironia_negativo           [DIFÍCIL   ]: 100.0% (16/16)

ironia_positivo           [DIFÍCIL   ]: 100.0% (11/11)

negacion_negativo         [MUY DIFÍCIL]: 100.0% (7/7)

negacion_positivo         [MUY DIFÍCIL]:  93.3% (14/15)

simple_negativo           [FÁCIL     ]: 100.0% (53/53)

simple_positivo           [FÁCIL     ]: 100.0% (49/49)

💡 OBSERVACIONES PEDAGÓGICAS

Como era de esperarse:

✓ CASOS SIMPLES: Alta precisión (~85-95%)
  El modelo funciona bien cuando las palabras coinciden con el sentimiento.

⚠ IRONÍA/SARCASMO: Precisión media-baja (~50-70%)
  El modelo se confunde porque las palabras no coinciden con el sentimiento real.

✗ NEGACIONES: Precisión variable
  "No funciona" vs "No tengo quejas" - ambas tienen "no", pero significan opuesto.

CONCLUSIÓN: Los modelos clásicos (TF-IDF + Logistic Regression) son un 
buen BASELINE, pe

### Análisis de Rendimiento por Tipo de Reseña

Veamos cómo le va al modelo en diferentes tipos de casos.

---

## 🧠 Guía Teórico-Conceptual

### 1. Flujo completo de un proyecto de clasificación de texto

**Paso 1: Recolección de datos**  
Obtener un corpus etiquetado (en nuestro caso, reseñas con sentimiento)

**Paso 2: Preprocesamiento**  
Limpiar texto, tokenización, opcional: stemming/lemmatización

**Paso 3: Vectorización**  
Convertir texto en representación numérica (BoW, TF-IDF, embeddings)

**Paso 4: División train/test**  
Separar datos para entrenar y evaluar sin sesgo

**Paso 5: Entrenamiento**  
Ajustar modelo a los datos de entrenamiento

**Paso 6: Evaluación**  
Medir rendimiento en datos de prueba

**Paso 7: Optimización**  
Ajustar hiperparámetros, probar otros modelos

**Paso 8: Despliegue**  
Usar el modelo en producción

---

### 2. Bag of Words vs. TF-IDF

**Bag of Words (BoW):**
- Representa documentos como vectores de conteos de palabras
- Ignora orden y gramática
- Simple pero efectivo como baseline
- **Problema**: Palabras muy frecuentes dominan la representación

**TF-IDF:**
- Balancea frecuencia local (documento) con rareza global (corpus)
- Palabras comunes reciben pesos bajos
- Palabras discriminativas reciben pesos altos
- Generalmente supera a BoW en clasificación de texto

---

### 3. Regresión Logística para Clasificación

**Funcionamiento:**
- Aprende función lineal: z = w₁x₁ + w₂x₂ + ... + wₙxₙ + b
- Aplica sigmoide: P(y=1|x) = 1 / (1 + e⁻ᶻ)
- Salida es probabilidad entre 0 y 1
- Si P > 0.5 → Clase 1, sino → Clase 0

**Ventajas:**
- Rápido y eficiente
- Probabilidades calibradas
- Pesos interpretables
- Funciona bien con alta dimensionalidad

**Limitaciones:**
- Asume relaciones lineales
- No captura interacciones complejas
- Ignora orden de palabras

---

### 4. Métricas de Evaluación

**Accuracy:** Porcentaje de predicciones correctas (cuidado con clases desbalanceadas)

**Precision:** De las predichas positivas, ¿cuántas son realmente positivas?

**Recall:** De las positivas reales, ¿cuántas detectamos?

**F1-Score:** Media armónica entre precision y recall

---

### 5. Importancia del Baseline

Antes de usar redes neuronales complejas, siempre establecemos un baseline simple para:

1. Entender la dificultad del problema
2. Detectar problemas en los datos
3. Tener punto de comparación cuantitativo
4. Justificar complejidad adicional
5. Iterar rápidamente

En muchos casos, un modelo simple bien ajustado es suficiente y preferible (más rápido, interpretable, fácil de mantener).

---

## ❓ Preguntas y Respuestas para Estudio

### Preguntas Conceptuales

**1. ¿Por qué es importante dividir los datos en conjuntos de entrenamiento y prueba?**

*Respuesta:* Para evaluar el rendimiento del modelo en datos que nunca vio durante el entrenamiento. Si evaluáramos con los mismos datos de entrenamiento, el modelo podría haber memorizado los ejemplos (overfitting) y no generalizar bien a datos nuevos del mundo real.

---

**2. ¿Cuál es la diferencia principal entre Bag of Words y TF-IDF?**

*Respuesta:* BoW solo cuenta la frecuencia de cada palabra en el documento, mientras que TF-IDF pondera esa frecuencia según qué tan rara es la palabra en todo el corpus. TF-IDF reduce la importancia de palabras muy comunes (como "el", "de") y aumenta la de palabras discriminativas.

---

**3. ¿Por qué usamos Regresión Logística para clasificación si su nombre dice "regresión"?**

*Respuesta:* Aunque el nombre es confuso, la Regresión Logística es un algoritmo de clasificación. Usa una función logística (sigmoide) para convertir una combinación lineal de features en una probabilidad entre 0 y 1, y luego clasifica según un umbral (típicamente 0.5).

---

**4. ¿Qué es el overfitting y cómo lo evitamos en este notebook?**

*Respuesta:* Overfitting ocurre cuando el modelo memoriza los datos de entrenamiento en lugar de aprender patrones generalizables. Lo evitamos mediante: (1) división train/test, (2) limitación del vocabulario (max_features=1000), y (3) regularización implícita en LogisticRegression.

---

**5. ¿Por qué nunca debemos usar fit() en los datos de prueba?**

*Respuesta:* Porque fit() aprende parámetros de los datos (vocabulario, escalas, etc.). Si lo usamos en datos de prueba, el modelo "espía" información que no debería conocer, invalidando la evaluación. Solo debemos usar transform() en datos nuevos.

---

### Preguntas Técnicas

**6. Si tenemos un dataset con 90% de reseñas positivas y 10% negativas, ¿qué problema tiene usar solo accuracy como métrica?**

*Respuesta:* Un modelo trivial que prediga "positivo" para todo tendría 90% de accuracy sin aprender nada útil. En datasets desbalanceados, debemos usar precision, recall y F1-score para evaluar el rendimiento en cada clase.

---

**7. ¿Qué significa que TfidfVectorizer devuelva una "matriz dispersa" (sparse matrix)?**

*Respuesta:* Como la mayoría de las entradas son cero (cada documento solo contiene una pequeña fracción del vocabulario total), scipy usa una representación dispersa que solo almacena los valores no-cero. Esto ahorra memoria y acelera cálculos.

---

**8. ¿Cómo interpretamos los coeficientes de la Regresión Logística?**

*Respuesta:* Coeficientes positivos indican que la presencia de esa palabra aumenta la probabilidad de clase positiva. Coeficientes negativos indican asociación con la clase negativa. La magnitud indica la fuerza de la asociación.

---

**9. ¿Qué pasaría si no usáramos random_state en train_test_split?**

*Respuesta:* Cada ejecución del notebook produciría una división diferente, resultando en métricas ligeramente distintas. Fijar random_state garantiza reproducibilidad, importante para debugging y comparación de modelos.

---

**10. ¿Por qué limitamos max_features a 1000 palabras?**

*Respuesta:* Para reducir dimensionalidad y evitar overfitting. Palabras muy raras (que aparecen en 1-2 documentos) suelen ser ruido o typos. Mantener solo las más frecuentes captura la mayor parte de la información con menor riesgo de sobreajuste.

---

### Preguntas de Aplicación

**11. Mencioná tres limitaciones del enfoque BoW/TF-IDF que las redes neuronales podrían superar.**

*Respuesta:*
1. Ignoran el orden de las palabras ("no es bueno" vs "es bueno")
2. No capturan significado semántico ("excelente" y "genial" son tratadas como completamente diferentes)
3. No modelan dependencias largas ni contexto complejo

---

**12. Si tuvieras que clasificar tweets (textos muy cortos), ¿qué ajustes harías a este enfoque?**

*Respuesta:*
- Incluir n-gramas (bigramas, trigramas) para capturar frases cortas
- Reducir max_features (menos palabras únicas en tweets)
- Considerar preprocesamiento de hashtags, menciones y emojis
- Probar con char-level features para manejar jerga y typos

---

**13. ¿En qué casos preferirías usar Regresión Logística sobre una red neuronal profunda?**

*Respuesta:*
- Dataset pequeño (pocas muestras)
- Necesidad de interpretabilidad (explicar decisiones)
- Recursos computacionales limitados
- Necesidad de entrenar/desplegar rápidamente
- Cuando el baseline ya da resultados satisfactorios

---

## 🎯 Ejercicios Propuestos

### Ejercicio 1: Experimentación con Hiperparámetros
Probá cambiar `max_features` en TfidfVectorizer a 500, 2000 y 5000. ¿Cómo afecta al accuracy? ¿Observás overfitting con vocabularios muy grandes?

### Ejercicio 2: N-gramas
Modificá TfidfVectorizer para incluir bigramas: `TfidfVectorizer(max_features=1000, ngram_range=(1,2))`. ¿Mejora el rendimiento? ¿Por qué los bigramas pueden ser útiles en español?

### Ejercicio 3: Dataset Completo
Probá entrenar con el dataset completo (200,000 reviews) en lugar del subset de 10,000. ¿Mejora significativamente el accuracy? ¿Cuánto tarda el entrenamiento?

### Ejercicio 4: Análisis de Errores
Identificá 5 reseñas del conjunto de prueba que el modelo clasificó incorrectamente. ¿Qué tienen en común? ¿Son casos difíciles incluso para humanos? (Tip: ironía, sarcasmo, negaciones)

### Ejercicio 5: Comparación de Modelos
Probá otros clasificadores de sklearn: `MultinomialNB`, `SVC`, `RandomForestClassifier`. ¿Cuál da mejores resultados en español? ¿Cuál es más rápido?

### Ejercicio 6: Dataset Propio
Buscá otro dataset en español (Twitter, reviews de apps, noticias) y aplicá el mismo pipeline. ¿El modelo generaliza bien a otros dominios?

---

## 🎓 Conclusión

En este notebook establecimos un baseline sólido para clasificación de sentimientos en español usando métodos clásicos de Machine Learning. Aprendimos:

1. ✅ Cargar datasets de HuggingFace con reviews reales en español
2. ✅ Preprocesar datos y convertir problemas multiclase a binarios
3. ✅ Vectorizar texto con BoW y TF-IDF
4. ✅ Entrenar modelos de Regresión Logística
5. ✅ Evaluar con métricas apropiadas
6. ✅ Interpretar qué palabras en español influyen en las predicciones

**Próximo paso:** En el siguiente notebook vamos a explorar Naive Bayes y Pipelines de sklearn para construir flujos de trabajo más modulares y profesionales.

---

### 💡 Reflexión Final

Este modelo clásico (TF-IDF + Logistic Regression) es sorprendentemente efectivo para clasificación de sentimientos. En muchos casos reales de la industria, un baseline bien ajustado como este es suficiente y preferible por su:

- ⚡ Velocidad de entrenamiento e inferencia
- 📊 Interpretabilidad (podemos explicar por qué clasifica así)
- 💻 Bajos requisitos computacionales
- 🔧 Facilidad de mantenimiento

Solo cuando este baseline no alcanza la performance requerida, justificamos la complejidad adicional de redes neuronales profundas.

---

*Este material fue desarrollado con fines educativos para la Tecnicatura en Ciencia de Datos del IFTS.*